In [8]:
using Revise
using GLMakie
include("EFITModule/EFIT.jl")
using NPZ

In [9]:
using ParallelStencil
using ParallelStencil.FiniteDifferences3D
using GLMakie

ParallelStencil.@reset_parallel_stencil()
USE_GPU = false
@static if USE_GPU
    @init_parallel_stencil(CUDA, Float32, 3)
else
    @init_parallel_stencil(Threads, Float32, 3)
end

In [10]:
matGrid = npzread("MeshFiles/IIWMini.npy")[:,:,1:90].+1


#Maximum sound speed in the model
cMax = 5918
cMin = 3225
#Frequency, hz
f0=2.25e6
#Period
t0 = 1.00 / f0
#Maximum spatial increment
dx = cMin/(8*f0)
#Maximum time increment
dt = dx/(cMax*sqrt(3))


LoadError: SystemError: opening file "MeshFiles/IIWMini.npy": No such file or directory

In [4]:
display(volume(matGrid.-1))

GLMakie.Screen(...)

In [5]:
materials = [Main.EFIT.IsoMat(0,0,1e-10), Main.EFIT.IsoMats["AISI1018"]]

2-element Vector{Main.EFIT.IsoMat}:
 Main.EFIT.IsoMat(1.0f-10, 0.0f0, 0.0f0, 0.0f0)
 Main.EFIT.IsoMat(7870.0f0, 1.11923f11, 8.1852916f10, 2.7562885f11)

In [6]:
grid = Main.EFIT.EFITGrid(Int16.(matGrid),materials,dt,dx);

sourcePositions = [(160:165,200:240,35:55)]

#Create a source function from which input can be taken
function source(t)
    v = exp(-((2*(t-2*t0)/(t0))^2))*sin(2*pi*f0*t)*0.1
    return Data.Number(v)
end
nframes = 600
framerate = 30

tIterator = 0:grid.dt:grid.dt*nframes
fig,ax,plt = volume(Array(grid.vx),algorithm=:mip,colorrange = (0, 0.1),colormap=:curl, transparency=true)
#Step the simulation
function stepSim(t)
    println(t)
    #Isotropic step function
    Main.EFIT.SimStep!(grid)
    #Apply the source to the Z-direction
    @parallel (sourcePositions[1]) Main.EFIT.applySource!(grid.vx,grid.vy,grid.vz, Data.Number(0.0), Data.Number(0.0), (source(t)))
    velMag = sqrt.(grid.vx.^2 .+ grid.vy.^2 .+ grid.vz.^2)
    #Update the plot
    plt.volume = velMag
end

record(stepSim, fig, "color_animation.mp4", tIterator; framerate = 30)


Creating grid of size 167, 497, 90

0.0
1.7479204e-8
3.4958408e-8
5.2437613e-8
6.9916815e-8
8.739602e-8
1.0487523e-7
1.2235442e-7
1.3983363e-7
1.5731284e-7
1.7479204e-7
1.9227124e-7
2.0975045e-7
2.2722965e-7
2.4470884e-7
2.6218805e-7
2.7966726e-7
2.9714647e-7
3.1462568e-7
3.3210486e-7
3.4958407e-7
3.6706328e-7
3.845425e-7
4.020217e-7
4.195009e-7
4.369801e-7
4.544593e-7
4.719385e-7
4.894177e-7
5.068969e-7
5.243761e-7
5.4185534e-7
5.593345e-7
5.768137e-7
5.9429294e-7
6.117721e-7
6.2925136e-7
6.4673054e-7
6.642097e-7
6.8168896e-7
6.9916814e-7
7.166474e-7
7.3412656e-7
7.5160574e-7
7.69085e-7
7.8656416e-7
8.040434e-7
8.215226e-7
8.390018e-7
8.56481e-7
8.739602e-7
8.914394e-7
9.089186e-7
9.2639783e-7
9.43877e-7
9.613562e-7
9.788354e-7
9.963146e-7
1.0137938e-6
1.031273e-6
1.0487522e-6
1.0662314e-6
1.0837107e-6
1.1011898e-6
1.118669e-6
1.1361483e-6
1.1536274e-6
1.1711066e-6
1.1885859e-6
1.2060651e-6
1.2235442e-6
1.2410235e-6
1.2585027e-6
1.2759818e-6
1.2934611e-6
1.3109403e-6
1.3284194e-6
1.3

LoadError: Screen not open!